## Importing the necessary modules


In [1]:
from json import loads
from random import choice, shuffle
from string import punctuation
from keras import Sequential
from keras.layers import Dense, Dropout, LeakyReLU, Softmax, Input
from keras.models import save_model
from keras.optimizers import Adam
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from numpy import array
'''
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')
'''


'\nnltk.download("punkt")\nnltk.download("wordnet")\nnltk.download(\'omw-1.4\')\n'

## Loading the dataset


In [2]:
data = loads(open("../dataset/intents.json", encoding="utf-8").read())


## Using NLP to clean the data


In [3]:
lemmatizer = WordNetLemmatizer()

# Each list to create
words = []  # tokenized words of sentences in patterns
classes = []  # tags
doc_X = []  # patterns
doc_y = []  # tags ocurring number of times wrt patterns

'''Looping through all the intents and tokenizing each patterns and
appending tokens to words, patterns and associated tag to
their associated list'''
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    # add the tag to the classes if it's not there already
    if intent["tag"] not in classes:
        classes.append(intent["tag"])


In [4]:
# lemmatizing all the words in the vocab and
# converting them to lowercase if the words
# don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower())
         for word in words if word not in punctuation]


In [5]:
'''sorting the vocab and classes in alphabetical order
and taking the set to ensure no duplicates occur'''
words = sorted(set(words))
classes = sorted(set(classes))


In [6]:
print("lenght of words: ", len(words))
print("length of doc_X", len(doc_X))
print("length of doc_y", len(doc_y))
print("length of classes", len(classes))


lenght of words:  173
length of doc_X 231
length of doc_y 231
length of classes 61


## Creating training and test sets


In [7]:
# list for training data
training = []

# creating the bag of words model
for idx, doc in enumerate(doc_X):
    bow = []
    # lemmatizing the sentence and converting to lowercase
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        # one hot encoding the words if they appear in the text
        if word in text:
            bow.append(1)
        else:
            bow.append(0)
    output_row = [0] * len(classes)
    # marking the index of class that the current pattern is associated to as 1
    output_row[classes.index(doc_y[idx])] = 1
    # adding the one hot encoded BoW and associated classes to training
    training.append([bow, output_row])


In [8]:
# shuffling the data and convert it to a numpy array
shuffle(training)
training = array(training, dtype=object)


In [9]:
# splitting the features and target labels
train_X = array(list(training[:, 0]))  # features
train_y = array(list(training[:, 1]))  # target labels


In [10]:
print(train_X)


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [11]:
print(train_y)


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [12]:
print(f"Shape of train_X: {train_X.shape}")
print(f"Shape of train_y: {train_y.shape}")


Shape of train_X: (231, 173)
Shape of train_y: (231, 61)


# Building deep learning model


In [13]:
# defining some parameters
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
print("input_shape: ", input_shape)
print("output_shape: ", output_shape)


input_shape:  (173,)
output_shape:  61


In [14]:
# defining the model
model = Sequential(name="chatKGEC")
model.add(Input(shape=input_shape, name="input_layer"))
model.add(Dense(256, activation=LeakyReLU(alpha=0.2), name="hidden_layer_1"))
model.add(Dropout(0.25, name="dropout_layer_1"))
model.add(Dense(128, activation=LeakyReLU(alpha=0.2), name="hidden_layer_2"))
model.add(Dropout(0.25, name="dropout_layer_2"))
model.add(Dense(64, activation=LeakyReLU(alpha=0.2), name="hidden_layer_3"))
model.add(Dropout(0.25, name="dropout_layer_3"))
model.add(Dense(output_shape, activation=Softmax(), name="output_layer"))


In [15]:
# compiling the model
adam = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=adam, metrics=["accuracy"])


In [16]:
print(model.summary())


Model: "chatKGEC"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden_layer_1 (Dense)      (None, 256)               44544     
                                                                 
 dropout_layer_1 (Dropout)   (None, 256)               0         
                                                                 
 hidden_layer_2 (Dense)      (None, 128)               32896     
                                                                 
 dropout_layer_2 (Dropout)   (None, 128)               0         
                                                                 
 hidden_layer_3 (Dense)      (None, 64)                8256      
                                                                 
 dropout_layer_3 (Dropout)   (None, 64)                0         
                                                                 
 output_layer (Dense)        (None, 61)                396

## Training the model


In [17]:
model.fit(x=train_X, y=train_y, epochs=200)  # validation_split=0.2


Epoch 1/200


2023-05-13 15:49:14.132034: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


8/8 [==============================] - 0s 2ms/step - loss: 4.0096 - accuracy: 0.0779
Epoch 2/200
8/8 [==============================] - 0s 2ms/step - loss: 3.1601 - accuracy: 0.1775
Epoch 3/200
8/8 [==============================] - 0s 2ms/step - loss: 2.2918 - accuracy: 0.3117
Epoch 4/200
8/8 [==============================] - 0s 2ms/step - loss: 1.6887 - accuracy: 0.5195
Epoch 5/200
8/8 [==============================] - 0s 2ms/step - loss: 1.1494 - accuracy: 0.6147
Epoch 6/200
8/8 [==============================] - 0s 2ms/step - loss: 0.8779 - accuracy: 0.7013
Epoch 7/200
8/8 [==============================] - 0s 2ms/step - loss: 0.7253 - accuracy: 0.7662
Epoch 8/200
8/8 [==============================] - 0s 2ms/step - loss: 0.6142 - accuracy: 0.7835
Epoch 9/200
8/8 [==============================] - 0s 2ms/step - loss: 0.5198 - accuracy: 0.8268
Epoch 10/200
8/8 [==============================] - 0s 2ms/step - loss: 0.3758 - accuracy: 0.8831
Epoch 11/200
8/8 [=======================

# Saving the model


In [18]:
save_model(model, "model.h5")
